<a href="https://colab.research.google.com/github/ANAVDUTT/IPHS290_CulturalAnalytics/blob/main/Copy_of_scrape_twitter_usgovs_simple_20221119.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REST URL ONLY: Scrape Twitter for US Governors in 2022 Mid-Terms

# Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import glob
import time
import requests

import json
from pprint import pprint

In [ ]:
import tweepy
import gc #collecting garbage
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator #visualisation

sns.set_theme(context='notebook', style='whitegrid', palette='vlag', font='serif')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Mount GDrive to save long-running API data collection results

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/courses/2022f_iphs220_culturalanalytics/twitter
!ls -altr

[Errno 2] No such file or directory: '/gdrive/MyDrive/courses/2022f_iphs220_culturalanalytics/twitter'
/content
total 2420
drwxr-xr-x 4 root root   4096 Nov 16 14:34 .config
drwxr-xr-x 1 root root   4096 Nov 16 14:35 sample_data
-rw-r--r-- 1 root root    505 Nov 18 12:35 config_twitter.py
-rw-r--r-- 1 root root   2687 Nov 18 12:35 twitter_2022midterm_us_governors.csv
drwxr-xr-x 2 root root   4096 Nov 18 12:36 __pycache__
-rw-r--r-- 1 root root 286562 Nov 18 12:41 twitter_all_follower_2244994945.csv
-rw-r--r-- 1 root root 286562 Nov 18 12:42 twitter_all_follower_20221118-124229.csv
-rw-r--r-- 1 root root 286562 Nov 18 12:43 twitter_all_follower_20221118-124356.csv
-rw-r--r-- 1 root root 286562 Nov 18 12:45 twitter_all_following_20221118-124533.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:46 twitter_following_2244994945.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:49 twitter_following_12345.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:49 twitter_following_23456.csv
-rw-r--r-- 1 root root 26

**VERIFY: You must have the following two files in this directory**

1. twitter_config.py (with OAuth credentials)

2. twitter_2022midterm_us_governors.csv (Rep/Dem governors with usernames to scrape)

# Get Data

**CURRENT DIR MUST HAVE: 'twitter_2022midterm_us_governors.csv'**

In [ ]:
govs_df = pd.read_csv('twitter_2022midterm_us_governors.csv', index_col=[0])
govs_df.head()
govs_df.info()

,Republican,Democrat
State,,
Wisconsin:,https://twitter.com/michelsforgov,https://twitter.com/GovEvers
Orgeon:,https://twitter.com/ChristineDrazan,https://twitter.com/TinaKotek
Nevada:,https://twitter.com/JoeLombardoNV,https://twitter.com/GovSisolak
Arizona:,https://twitter.com/KariLake,https://twitter.com/katiehobbs
Kansas:,https://twitter.com/DerekSchmidtKS,https://twitter.com/LauraKellyKS


<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, Wisconsin: to Idaho:
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Republican  35 non-null     object
 1   Democrat    35 non-null     object
dtypes: object(2)
memory usage: 840.0+ bytes


In [ ]:
# Clean imported DataFrame

# Lowercase Columns
govs_df.rename(columns={'Republican':'republican', 'Democrat':'democrat'}, inplace=True)


# Move State Index to separate colum for joining
state_orig_ls = govs_df.index.to_list()
# state_orig_ls[:5]

state_clean_ls = [x.lower().strip(':') for x in state_orig_ls]
# state_clean_ls[:5]

# Insert state col in first position
govs_df.insert(loc=0, column='state', value=state_clean_ls)
# govs_df['state'] = state_clean_ls

# Drop the State Index
govs_df.reset_index(drop=True, inplace=True)

govs_df.head()


,state,republican,democrat
0,wisconsin,https://twitter.com/michelsforgov,https://twitter.com/GovEvers
1,orgeon,https://twitter.com/ChristineDrazan,https://twitter.com/TinaKotek
2,nevada,https://twitter.com/JoeLombardoNV,https://twitter.com/GovSisolak
3,arizona,https://twitter.com/KariLake,https://twitter.com/katiehobbs
4,kansas,https://twitter.com/DerekSchmidtKS,https://twitter.com/LauraKellyKS


In [ ]:
# Split into Two (1) Dem and (2) Rep tables

govs_d_df = govs_df[['state','democrat']].copy(deep=True)
# Extract username from url
govs_d_df['screen_name'] = govs_df["democrat"].apply(lambda x: str(x).split('/')[-1])
govs_d_df.head()

govs_r_df = govs_df[['state','republican']].copy(deep=True) 
# Extract username from url
govs_r_df['screen_name'] = govs_df["republican"].apply(lambda x: str(x).split('/')[-1])
govs_r_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


,state,republican,screen_name
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV
3,arizona,https://twitter.com/KariLake,KariLake
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS


# Get OAuth Tokens

**CURRENT DIR MUST HAVE: 'config_twitter.py'**

In [ ]:
# If you want to hardcode your secret in this Jupyter notebook

# bearer_token = 'Thisismycrazylongtwitter-bearingtokenetex' # Your API/Bearer Token

In [ ]:
import config_twitter

api_key = config_twitter.API_KEY #Your API/Consumer key 
api_secret = config_twitter.API_KEY_SECRET #Your API/Consumer Secret Key
bearer_token = config_twitter.BEARER_TOKEN # Your API/Bearer Token
access_token = config_twitter.ACCESS_TOKEN    #Your Access token key
access_token_secret = config_twitter.ACCESS_TOKEN_SECRET #Your Access token Secret key

# REST URI Endpoints (v2.0 API)

Regular 

* https://github.com/twitterdev/Twitter-API-v2-sample-code

Academic

* https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/3-deciding-which-endpoints-to-use.md

In [ ]:
%whos DataFrame

Variable    Type         Data/Info
----------------------------------
govs_d_df   DataFrame                 state       <...>dtforgov      heidtforgov
govs_df     DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df   DataFrame                 state       <...>alABundy       RealABundy


In [ ]:
# COPY/RENAME dataframes (removed Tweepy code that did this before)

govs_d_user_df = govs_d_df.copy()
govs_r_user_df = govs_r_df.copy()

In [ ]:
govs_d_user_df.head()
govs_d_user_df.info()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   state        35 non-null     object
 1   democrat     35 non-null     object
 2   screen_name  35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


## Common Functions

In [ ]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

In [ ]:
def connect_to_endpoint_wo_params(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [ ]:
def connect_to_endpoint_with_params(url, tweet_fields):
    response = requests.request(
        "GET", url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [ ]:
def get_followers(user_id):
    url = "https://api.twitter.com/2/users/{}/following".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test
user_id = 234896532
json_response = get_followers(user_id)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_followers_df = pd.DataFrame(json_response['data'])
my_followers_df.head()
my_followers_df.info()
    

200
{
    "data": [
        {
            "created_at": "2021-05-04T19:31:07.000Z",
            "id": "1389663738945495040",
            "name": "Dr. Jill Underly, WI State Superintendent",
            "username": "DrJillUnderly"
        },
        {
            "created_at": "2021-01-06T17:02:07.000Z",
            "id": "1346864195304792065",
            "name": "Senator Brad Pfaff",
            "username": "SenBradPfaff"
        },
        {
            "created_at": "2009-05-27T19:35:11.000Z",
            "id": "42953875",
            "name": "Western Tech College",
            "username": "WesternTC"
        },
        {
            "created_at": "2008-07-21T18:48:42.000Z",
            "id": "15518969",
            "name": "Ducks Unlimited",
            "username": "DucksUnlimited"
        },
        {
            "created_at": "2018-05-01T14:15:25.000Z",
            "id": "991320189634478080",
            "name": "Forward Madison FC",
            "username": "ForwardMSNFC"
       

,id,name,username,created_at
0,1389663738945495040,"Dr. Jill Underly, WI State Superintendent",DrJillUnderly,2021-05-04T19:31:07.000Z
1,1346864195304792065,Senator Brad Pfaff,SenBradPfaff,2021-01-06T17:02:07.000Z
2,42953875,Western Tech College,WesternTC,2009-05-27T19:35:11.000Z
3,15518969,Ducks Unlimited,DucksUnlimited,2008-07-21T18:48:42.000Z
4,991320189634478080,Forward Madison FC,ForwardMSNFC,2018-05-01T14:15:25.000Z


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          100 non-null    object
 1   name        100 non-null    object
 2   username    100 non-null    object
 3   created_at  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [ ]:
def get_user_by_username(username, param_str):

    url = f'https://api.twitter.com/2/users/by/username/{username}?user.fields={param_str}'
    # url = f"https://api.twitter.com/2/users/by?user.fields={params}"
    params = {"user.fields": param_str}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test

username = 'KariLake'
params = 'created_at,description' # ,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

user_id = 234896532
json_response = get_user_by_username(username, params)
print(json.dumps(json_response, indent=4, sort_keys=True))

400


Exception: ignored

In [ ]:
def get_user_by_username(username, param_str):

    url = f'https://api.twitter.com/2/users/by/username/{username}?user.fields={param_str}'
    # url = f"https://api.twitter.com/2/users/by?user.fields={params}"
    params = {"user.fields": param_str}
    json_response = connect_to_endpoint_wo_params(url) # , params)
    # json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test

username = 'KariLake'
params = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

# user_id = 234896532
json_response = get_user_by_username(username, params)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_followers_df = pd.DataFrame(pd.json_normalize(json_response['data']))
my_followers_df.head()
my_followers_df.info()



url = 'https://api.twitter.com/2/users/by?user.fields='
url


200
{
    "data": {
        "created_at": "2013-01-12T07:47:51.000Z",
        "description": "Official Twitter Account for Trump-Endorsed Candidate for Arizona Governor. Text KARI to 70789. Find me on TRUTH Social at karilake & FB at TheKariLake.",
        "entities": {
            "url": {
                "urls": [
                    {
                        "display_url": "KariLake.com",
                        "end": 23,
                        "expanded_url": "http://KariLake.com",
                        "start": 0,
                        "url": "https://t.co/EmzgORJKeQ"
                    }
                ]
            }
        },
        "id": "1082197856",
        "location": "Iowa & Arizona",
        "name": "Kari Lake",
        "pinned_tweet_id": "1575893306487103498",
        "profile_image_url": "https://pbs.twimg.com/profile_images/1560073130822885378/55oLUgB0_normal.jpg",
        "protected": false,
        "public_metrics": {
            "followers_count": 715357,


,url,name,id,pinned_tweet_id,verified,profile_image_url,description,protected,username,location,created_at,entities.url.urls,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,https://t.co/EmzgORJKeQ,Kari Lake,1082197856,1575893306487103498,True,https://pbs.twimg.com/profile_images/156007313...,Official Twitter Account for Trump-Endorsed Ca...,False,KariLake,Iowa & Arizona,2013-01-12T07:47:51.000Z,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",715357,974,24431,1362


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   url                             1 non-null      object
 1   name                            1 non-null      object
 2   id                              1 non-null      object
 3   pinned_tweet_id                 1 non-null      object
 4   verified                        1 non-null      bool  
 5   profile_image_url               1 non-null      object
 6   description                     1 non-null      object
 7   protected                       1 non-null      bool  
 8   username                        1 non-null      object
 9   location                        1 non-null      object
 10  created_at                      1 non-null      object
 11  entities.url.urls               1 non-null      object
 12  public_metrics.followers_count  1 non-null      int64 

'https://api.twitter.com/2/users/by?user.fields='

## Get User Info

In [ ]:
# Define DataFrame columns for all the User data elements we want to collect

govs_d_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

govs_r_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

In [ ]:
govs_d_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


In [ ]:
# Test

username = 'KariLake'

# user_id = 234896532
json_response = get_user_by_username(username, params)
print(json.dumps(json_response, indent=4, sort_keys=True))



200
{
    "data": {
        "created_at": "2013-01-12T07:47:51.000Z",
        "description": "Official Twitter Account for Trump-Endorsed Candidate for Arizona Governor. Text KARI to 70789. Find me on TRUTH Social at karilake & FB at TheKariLake.",
        "entities": {
            "url": {
                "urls": [
                    {
                        "display_url": "KariLake.com",
                        "end": 23,
                        "expanded_url": "http://KariLake.com",
                        "start": 0,
                        "url": "https://t.co/EmzgORJKeQ"
                    }
                ]
            }
        },
        "id": "1082197856",
        "location": "Iowa & Arizona",
        "name": "Kari Lake",
        "pinned_tweet_id": "1575893306487103498",
        "profile_image_url": "https://pbs.twimg.com/profile_images/1560073130822885378/55oLUgB0_normal.jpg",
        "protected": false,
        "public_metrics": {
            "followers_count": 715359,


In [ ]:
# Get User data for each Dem Governor

govs_d_user_df = pd.DataFrame()

params = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

i = 0
for i,agov_username in enumerate(govs_d_df.screen_name.to_list()):
    print(f'Processing #{i}: {agov_username}')
    try:
        json_response = get_user_by_username(agov_username, params)
        # print(json.dumps(json_response, indent=4, sort_keys=True))

        my_userdata_df = pd.DataFrame(pd.json_normalize(json_response['data']))
        
        print(f'    Appending row: {my_userdata_df.head()}\n\n')
        
        if i == 0:
            govs_d_user_df = my_userdata_df.copy(deep=True)
        else:
            govs_d_user_df = pd.concat([govs_d_user_df, my_userdata_df], ignore_index=True)                

        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')
    

    i += 1



Processing #0: GovEvers
200
    Appending row:                        url                                        description  \
0  https://t.co/JyH2JWfeyt  Husband to Kathy, lifelong educator, Euchre an...   

          id  verified  protected                created_at  \
0  234896532      True      False  2011-01-06T21:05:10.000Z   

                                   profile_image_url                 name  \
0  https://pbs.twimg.com/profile_images/136168111...  Governor Tony Evers   

             location  username      pinned_tweet_id  \
0  Madison, Wisconsin  GovEvers  1590413394724913152   

                                   entities.url.urls  \
0  [{'start': 0, 'end': 23, 'url': 'https://t.co/...   

   public_metrics.followers_count  public_metrics.following_count  \
0                          106302                             244   

   public_metrics.tweet_count  public_metrics.listed_count  
0                        7970                          906  


Processing #1: Tina

In [ ]:
govs_d_user_df.head()

,url,description,id,verified,protected,created_at,profile_image_url,name,location,username,pinned_tweet_id,entities.url.urls,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,entities.description.hashtags,entities.description.mentions,entities.description.urls
0,https://t.co/JyH2JWfeyt,"Husband to Kathy, lifelong educator, Euchre an...",234896532,True,False,2011-01-06T21:05:10.000Z,https://pbs.twimg.com/profile_images/136168111...,Governor Tony Evers,"Madison, Wisconsin",GovEvers,1590413394724913152,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",106302,244,7970,906,NaN,NaN,NaN
1,https://t.co/U0kpZIDyUZ,Progressive Democrat. Candidate for Oregon Gov...,2835931010,True,False,2014-09-30T18:28:02.000Z,https://pbs.twimg.com/profile_images/150087938...,Tina Kotek,Oregon,TinaKotek,1590555133624348672,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",26614,338,1635,307,NaN,NaN,NaN
2,https://t.co/9iq0pinVZs,Governor of Nevada. Proud Dad and Husband. Sta...,1080586433088258048,True,False,2019-01-02T22:07:36.000Z,https://pbs.twimg.com/profile_images/139958353...,Governor Sisolak,"Nevada, USA",GovSisolak,1575878545829568513,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",100647,123,12652,602,"[{'start': 98, 'end': 114, 'tag': '3MillionRea...",NaN,NaN
3,https://t.co/otdUvA8bmj,"Social worker, mom, wife, Arizona’s Secretary ...",26659619,True,False,2009-03-26T01:55:21.000Z,https://pbs.twimg.com/profile_images/159335670...,Katie Hobbs,"Phoenix, AZ",katiehobbs,1592343416981303296,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",168391,1581,10029,926,NaN,NaN,NaN
4,https://t.co/ErZXA4bB3F,"48th Governor of Kansas, Wife, Mother, Champio...",364864453,True,False,2011-08-30T13:39:23.000Z,https://pbs.twimg.com/profile_images/154147570...,Laura Kelly,"Topeka, KS",LauraKellyKS,1590388380399198208,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",55468,555,2781,390,NaN,NaN,NaN


In [ ]:
# Test

# Specify the usernames that you want to lookup below
# You can enter up to 100 comma-separated values.
usernames = "usernames=TwitterDev,TwitterAPI"
user_fields = "user.fields=description,created_at"
# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    
json_response = connect_to_endpoint_wo_params(url)
print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": [
        {
            "created_at": "2013-12-14T04:35:55.000Z",
            "description": "The voice of the #TwitterDev team and your official source for updates, news, and events, related to the #TwitterAPI.",
            "id": "2244994945",
            "name": "Twitter Dev",
            "username": "TwitterDev"
        },
        {
            "created_at": "2007-05-23T06:01:13.000Z",
            "description": "Tweets about changes and service issues. Follow @TwitterDev\u00a0for more.",
            "id": "6253282",
            "name": "Twitter API",
            "username": "TwitterAPI"
        }
    ]
}


## Get Followers

Rate Limit: ? Per App/? Per User (within each 15min window)

(see https://developer.twitter.com/en/docs/twitter-api/rate-limits)

In [ ]:
%whos dict

Variable        Type    Data/Info
---------------------------------
json_response   dict    n=1


In [ ]:
# Test
user_id = 2244994945

url = "https://api.twitter.com/2/users/{}/followers".format(user_id)
params = {"user.fields": "created_at"}
json_response = connect_to_endpoint_with_params(url, params)

followers_df = pd.json_normalize(json_response['data'])
followers_df['destination'] = user_id
followers_ct = followers_df.shape[0]
followers_df.head()
followers_df.info()

200


,created_at,username,name,id,destination
0,2021-08-05T21:10:54.000Z,alwayscreatingg,Always Creating,1423391044855640068,2244994945
1,2017-06-29T15:16:04.000Z,hamtk9i9t0u7elu,sabarloder,880444762221826048,2244994945
2,2022-11-16T09:35:16.000Z,Nithiya80161237,Nithiyananthan A,1592813385099395080,2244994945
3,2017-07-07T13:10:39.000Z,jamesjdougherty,James Dougherty,883312305588535297,2244994945
4,2022-09-01T21:21:53.000Z,dehiakhlleil717,Dehia Asaeed,1565449386175483904,2244994945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   created_at   100 non-null    object
 1   username     100 non-null    object
 2   name         100 non-null    object
 3   id           100 non-null    object
 4   destination  100 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [ ]:
def get_followers(user_id):
    url = "https://api.twitter.com/2/users/{}/followers".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    
    followers_df = pd.json_normalize(json_response['data'])
    followers_ct = followers_df.shape[0]

    try:
        page_next_token = json_response['meta']['next_token']
    except:
        pass
    
    if not (page_next_token is None):
        
        while not (page_next_token is None):
            try:
                params = {"user.fields": "created_at", "pagination_token": page_next_token}
                json_response = connect_to_endpoint_with_params(url, params)
                my_followers_df = pd.json_normalize(json_response['data'])
                
                followers_df = pd.concat([followers_df, my_followers_df], ignore_index=True)
                followers_ct += len(json_response['data'])
                print(f'Current users_ct: {followers_ct}')
                
                try:
                    page_next_token = json_response['meta']['next_token']
                except:
                    pass
                      
            except:
                page_next_token = None
    
    print(f'Final users_ct: {followers_df}')
    
    return followers_df

### Run ONCE for EACH Governor

In [ ]:
# RESET user_id for each Governor and Run to END OF LOOP

gov_id = 2244994945


govs_d_followers_df = get_followers(gov_id)
govs_d_followers_df['source'] = govs_d_followers_df['id'] # Keep Original 'id' to remind all other fields are associated with Source (not Destination)
govs_d_followers_df['destination'] = gov_id
govs_d_followers_df['username'] = govs_d_followers_df['username'].astype('string')
govs_d_followers_df['username'] = govs_d_followers_df['username'].str.strip()
govs_d_followers_df['name'] = govs_d_followers_df['name'].astype('string')
govs_d_followers_df['name'] = govs_d_followers_df['name'].str.strip()
govs_d_followers_df.head()
govs_d_followers_df.info()

200
200
Current users_ct: 200
200
Current users_ct: 300
200
Current users_ct: 400
200
Current users_ct: 500
200
Current users_ct: 600
200
Current users_ct: 700
200
Current users_ct: 800
200
Current users_ct: 900
200
Current users_ct: 1000
200
Current users_ct: 1100
200
Current users_ct: 1200
200
Current users_ct: 1300
200
Current users_ct: 1400
429
Final users_ct:              username              name                   id  \
0     alwayscreatingg   Always Creating  1423391044855640068   
1     hamtk9i9t0u7elu        sabarloder   880444762221826048   
2     Nithiya80161237  Nithiyananthan A  1592813385099395080   
3     jamesjdougherty   James Dougherty   883312305588535297   
4     dehiakhlleil717      Dehia Asaeed  1565449386175483904   
...               ...               ...                  ...   
1395       ClimateFlo    Florian Börgel           3378671883   
1396  Anamari26469532  Satoshi Nakamoto  1558252208336125957   
1397   rowan_________    rowan schaefer  1297948611721007

,username,name,id,created_at,destination
0,alwayscreatingg,Always Creating,1423391044855640068,2021-08-05T21:10:54.000Z,2244994945
1,hamtk9i9t0u7elu,sabarloder,880444762221826048,2017-06-29T15:16:04.000Z,2244994945
2,Nithiya80161237,Nithiyananthan A,1592813385099395080,2022-11-16T09:35:16.000Z,2244994945
3,jamesjdougherty,James Dougherty,883312305588535297,2017-07-07T13:10:39.000Z,2244994945
4,dehiakhlleil717,Dehia Asaeed,1565449386175483904,2022-09-01T21:21:53.000Z,2244994945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   username     1400 non-null   string
 1   name         1400 non-null   string
 2   id           1400 non-null   object
 3   created_at   1400 non-null   object
 4   destination  1400 non-null   int64 
dtypes: int64(1), object(2), string(2)
memory usage: 54.8+ KB


In [ ]:
# Check not just repeating every 100

govs_d_followers_df.iloc[0:10]
govs_d_followers_df.iloc[95:105]

,username,name,id,created_at,destination,source
0,alwayscreatingg,Always Creating,1423391044855640068,2021-08-05T21:10:54.000Z,2244994945,1423391044855640068
1,hamtk9i9t0u7elu,sabarloder,880444762221826048,2017-06-29T15:16:04.000Z,2244994945,880444762221826048
2,Nithiya80161237,Nithiyananthan A,1592813385099395080,2022-11-16T09:35:16.000Z,2244994945,1592813385099395080
3,jamesjdougherty,James Dougherty,883312305588535297,2017-07-07T13:10:39.000Z,2244994945,883312305588535297
4,dehiakhlleil717,Dehia Asaeed,1565449386175483904,2022-09-01T21:21:53.000Z,2244994945,1565449386175483904
5,muhamuttaqien,Muh. Angga Muttaqien,385898085,2011-10-06T10:05:01.000Z,2244994945,385898085
6,MemoMogollan,Memo🦖,410513857,2011-11-12T06:05:49.000Z,2244994945,410513857
7,littleastr0,jonesey,158442862,2010-06-22T18:16:50.000Z,2244994945,158442862
8,stevenobx,Steve Parrish🇺🇸🍊🇺🇸,231586103,2010-12-28T23:09:48.000Z,2244994945,231586103
9,MatthewCKrueger,Matthew Krueger,396989791,2011-10-24T02:29:18.000Z,2244994945,396989791


,username,name,id,created_at,destination,source
95,georgemkhonazi8,George Mkhonazi,1552587152869007360,2022-07-28T09:30:29.000Z,2244994945,1552587152869007360
96,ToastBeb,Toast B,1593144458245074944,2022-11-17T07:30:40.000Z,2244994945,1593144458245074944
97,arittra_bag,Arittra Bag,1335611469132546053,2020-12-06T15:46:23.000Z,2244994945,1335611469132546053
98,Askareth,Askareth,402194945,2011-10-31T17:59:49.000Z,2244994945,402194945
99,OlavHjertaker,Olav Hjertaker,21333577,2009-02-19T20:07:39.000Z,2244994945,21333577
100,infj_eman,Eman 💭,2997813948,2015-01-27T04:59:43.000Z,2244994945,2997813948
101,LoriLesleyKT,Lori Lesley Martin,1483204767799287811,2022-01-17T22:29:45.000Z,2244994945,1483204767799287811
102,crowleydamion13,Damion Crowley,1578719916215222272,2022-10-08T12:13:27.000Z,2244994945,1578719916215222272
103,Malivipul61,Egoria Mfer,883220607508283392,2017-07-07T07:06:17.000Z,2244994945,883220607508283392
104,mercadodiego,Diego Mercado,156311191,2010-06-16T16:03:38.000Z,2244994945,156311191


In [ ]:
govs_d_followers_df.tail()

,username,name,id,created_at,destination,source
1395,ClimateFlo,Florian Börgel,3378671883,2015-07-16T12:28:12.000Z,2244994945,3378671883
1396,Anamari26469532,Satoshi Nakamoto,1558252208336125957,2022-08-13T00:41:26.000Z,2244994945,1558252208336125957
1397,rowan_________,rowan schaefer,1297948611721007104,2020-08-24T17:27:35.000Z,2244994945,1297948611721007104
1398,tasneem9013,tasneem ahmed,2900089884,2014-11-15T08:11:14.000Z,2244994945,2900089884
1399,FunnyMo00603145,FunnyMonkey,1562020337998409728,2022-08-23T10:14:16.000Z,2244994945,1562020337998409728


In [ ]:
# Verify

govs_d_followers_df.head()
govs_d_followers_df.info()

,username,name,id,created_at,destination,source
0,alwayscreatingg,Always Creating,1423391044855640068,2021-08-05T21:10:54.000Z,2244994945,1423391044855640068
1,hamtk9i9t0u7elu,sabarloder,880444762221826048,2017-06-29T15:16:04.000Z,2244994945,880444762221826048
2,Nithiya80161237,Nithiyananthan A,1592813385099395080,2022-11-16T09:35:16.000Z,2244994945,1592813385099395080
3,jamesjdougherty,James Dougherty,883312305588535297,2017-07-07T13:10:39.000Z,2244994945,883312305588535297
4,dehiakhlleil717,Dehia Asaeed,1565449386175483904,2022-09-01T21:21:53.000Z,2244994945,1565449386175483904


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   username     1400 non-null   string
 1   name         1400 non-null   string
 2   id           1400 non-null   object
 3   created_at   1400 non-null   object
 4   destination  1400 non-null   int64 
 5   source       1400 non-null   object
dtypes: int64(1), object(3), string(2)
memory usage: 65.8+ KB


In [ ]:
# Save

filename_follower_out = f'twitter_follower_{user_id}.csv'
govs_d_followers_df.to_csv(filename_follower_out, index=False)

**END OF LOOP**

In [ ]:
!ls -al twitter_follower_*.csv

-rw-r--r-- 1 root root 133868 Nov 18 13:31 twitter_follower_2244994945.csv


In [ ]:
# If Testing: Create dummy twitter_follower_*.csv copies to test Concatenation

!cp $filename_follower_out twitter_follower_12345.csv

!cp $filename_follower_out twitter_follower_23456.csv

!ls -al twitter_following_*.csv

-rw-r--r-- 1 root root 102186 Nov 18 12:49 twitter_following_12345.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:46 twitter_following_2244994945.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:49 twitter_following_23456.csv


### FINALLY: Concatenate all Followers DataFrames and Save to ONE *.csv File

In [ ]:
datafile_csv_ls = glob.glob('twitter_follower_*.csv')

all_df = pd.DataFrame()

for i, adatafile in enumerate(datafile_csv_ls):
    print(f'Processing #{i}: {adatafile}')
    current_df = pd.read_csv(adatafile, index_col=False)
    print(f'      shape: {current_df.shape}')
    if i == 0:
        all_df = current_df.copy(deep=True)
    else:
        all_df = pd.concat([all_df, current_df], ignore_index=True)
        
print('\n\n')
all_orig_ct = all_df.shape[0]
all_df.drop_duplicates(inplace=True)
all_nodup_ct = all_df.shape[0]
all_df.head()
all_df.info()
print(f'Combined DataFrame.shape: {all_df.shape} (dropped {all_orig_ct - all_nodup_ct} duplicate rows)')
    

Processing #0: twitter_follower_12345.csv
      shape: (1400, 6)
Processing #1: twitter_follower_2244994945.csv
      shape: (1400, 6)
Processing #2: twitter_follower_23456.csv
      shape: (1400, 6)





,username,name,id,created_at,destination,source
0,alwayscreatingg,Always Creating,1423391044855640068,2021-08-05T21:10:54.000Z,2244994945,1423391044855640068
1,hamtk9i9t0u7elu,sabarloder,880444762221826048,2017-06-29T15:16:04.000Z,2244994945,880444762221826048
2,Nithiya80161237,Nithiyananthan A,1592813385099395080,2022-11-16T09:35:16.000Z,2244994945,1592813385099395080
3,jamesjdougherty,James Dougherty,883312305588535297,2017-07-07T13:10:39.000Z,2244994945,883312305588535297
4,dehiakhlleil717,Dehia Asaeed,1565449386175483904,2022-09-01T21:21:53.000Z,2244994945,1565449386175483904


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   username     1400 non-null   object
 1   name         1400 non-null   object
 2   id           1400 non-null   int64 
 3   created_at   1400 non-null   object
 4   destination  1400 non-null   int64 
 5   source       1400 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 76.6+ KB
Combined DataFrame.shape: (1400, 6) (dropped 2800 duplicate rows)


In [ ]:
# Save Combined DataFrame

datetime_str = time.strftime("%Y%m%d-%H%M%S")

file_all_follower_str = f'twitter_all_follower_{datetime_str}.csv'

try:
  all_df.to_csv(file_all_follower_str, index=False)
  print(f'Saved to: {file_all_follower_str}')
except:
  print(f'ERROR: Cannot save to: {file_all_follower_str}')

Saved to: twitter_all_follower_20221118-133912.csv


In [ ]:
# Verify

!head -n 10 $file_all_follower_str

username,name,id,created_at,destination,source
alwayscreatingg,Always Creating,1423391044855640068,2021-08-05T21:10:54.000Z,2244994945,1423391044855640068
hamtk9i9t0u7elu,sabarloder,880444762221826048,2017-06-29T15:16:04.000Z,2244994945,880444762221826048
Nithiya80161237,Nithiyananthan A,1592813385099395080,2022-11-16T09:35:16.000Z,2244994945,1592813385099395080
jamesjdougherty,James Dougherty,883312305588535297,2017-07-07T13:10:39.000Z,2244994945,883312305588535297
dehiakhlleil717,Dehia Asaeed,1565449386175483904,2022-09-01T21:21:53.000Z,2244994945,1565449386175483904
muhamuttaqien,Muh. Angga Muttaqien,385898085,2011-10-06T10:05:01.000Z,2244994945,385898085
MemoMogollan,Memo🦖,410513857,2011-11-12T06:05:49.000Z,2244994945,410513857
littleastr0,jonesey,158442862,2010-06-22T18:16:50.000Z,2244994945,158442862
stevenobx,Steve Parrish🇺🇸🍊🇺🇸,231586103,2010-12-28T23:09:48.000Z,2244994945,231586103


### Expand ALL Followers

In [ ]:
%whos DataFrame

Variable                  Type         Data/Info
------------------------------------------------
all_df                    DataFrame                 username    <...>\n[1400 rows x 6 columns]
current_df                DataFrame                 username    <...>\n[1400 rows x 6 columns]
followers_df              DataFrame                      created<...>n\n[100 rows x 5 columns]
govs_d_df                 DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_followers_df       DataFrame                 username    <...>\n[1400 rows x 6 columns]
govs_d_following_df       DataFrame              source         <...>\n[1400 rows x 6 columns]
govs_d_user_df            DataFrame                            u<...>                    NaN  
govs_df                   DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df                 DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df            DataFrame    Empty Da

In [ ]:
# !ls twitter_all_follower_*.csv

!ls -al $file_all_follower_str

-rw-r--r-- 1 root root 133868 Nov 18 13:39 twitter_all_follower_20221118-133912.csv


In [ ]:
# CONFIGURE twitter_all_follower_csv to point to correct *.csv file of all merged followers

# twitter_all_follower_csv = 'twitter_all_follower_20221118-124356.csv'
twitter_all_follower_csv = file_all_follower_str

twitter_all_follower_df = pd.read_csv(twitter_all_follower_csv, index_col=False)
twitter_all_follower_df['username'] = twitter_all_follower_df['username'].astype('string')
twitter_all_follower_df['username'] = twitter_all_follower_df['username'].str.strip()
twitter_all_follower_df['name'] = twitter_all_follower_df['username'].astype('string')
twitter_all_follower_df['name'] = twitter_all_follower_df['username'].str.strip()
twitter_all_follower_df.head()
twitter_all_follower_df.info()

,username,name,id,created_at,destination,source
0,alwayscreatingg,alwayscreatingg,1423391044855640068,2021-08-05T21:10:54.000Z,2244994945,1423391044855640068
1,hamtk9i9t0u7elu,hamtk9i9t0u7elu,880444762221826048,2017-06-29T15:16:04.000Z,2244994945,880444762221826048
2,Nithiya80161237,Nithiya80161237,1592813385099395080,2022-11-16T09:35:16.000Z,2244994945,1592813385099395080
3,jamesjdougherty,jamesjdougherty,883312305588535297,2017-07-07T13:10:39.000Z,2244994945,883312305588535297
4,dehiakhlleil717,dehiakhlleil717,1565449386175483904,2022-09-01T21:21:53.000Z,2244994945,1565449386175483904


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   username     1400 non-null   string
 1   name         1400 non-null   string
 2   id           1400 non-null   int64 
 3   created_at   1400 non-null   object
 4   destination  1400 non-null   int64 
 5   source       1400 non-null   int64 
dtypes: int64(3), object(1), string(2)
memory usage: 65.8+ KB


#### Get User Details

In [ ]:
# Iteratively get User data for each Dem Governor


twitter_api2_delay_min = 15


# Define which info to retrieve for each user
params = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'


all_user_df = pd.DataFrame()


# Create a list of all UNIQUE Follower IDs (Source)
all_follower_username_ls = list(set(twitter_all_follower_df.username))
print(f'There are #{len(all_follower_username_ls)} Followers\n\n')
print(f'  First 5: {all_follower_username_ls[:5]}\n=====================================\n\n')


i = 0
# for i, afollower_username in enumerate(all_follower_username_ls):    # Get all follower followers 
for i, afollower_username in enumerate(all_follower_username_ls[:3]):   # Test first 3 followers
  id_cur = twitter_all_follower_df[twitter_all_follower_df['username']==afollower_username]['id'].iloc[0]
  username_cur = afollower_username
  index_cur = twitter_all_follower_df[twitter_all_follower_df['username']==afollower_username]['id'].index[0]

  print(f"Processing #{i}:\n  id: {id_cur}\n  username: {username_cur}\n  index: {index_cur}")

  try:
    # TODO: Add pagination beyond 1500 
    json_response = get_user_by_username(afollower_username, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))

    temp_user_df = pd.DataFrame(pd.json_normalize(json_response['data']))
    
    # print(f'    Appending row: {temp_user_df.head()}\n\n')
    
    if i == 0:
      all_user_df = temp_user_df.copy(deep=True)
    else:
      all_user_df = pd.concat([all_user_df, temp_user_df], ignore_index=True)                

    # Rename Columns
    all_user_df.rename(columns={'public_metrics.followers_count': 'followers_count', 
                                'public_metrics.following_count': 'following_count',
                                'public_metrics.tweet_count': 'tweet_count',
                                'public_metrics.listed_count': 'listed_count',
                                }, inplace=True)

    # Save
    filename_follower_out = f'twitter_expand_follower_{id_cur}.csv'
    all_user_df.to_csv(filename_follower_out, index=False)

    # Delay for Twitter API v2 limit of 1500 requests/15min period (+15 sec buffer)
    delay_sec = twitter_api2_delay_min*60 + 15
    print(f'  SLEEPING: {twitter_api2_delay_min} minutes (Twitter API v2 limit)')
    time.sleep(delay_sec)
      
  except:
    print(f'    ERROR: Could not retrieve Tweepy User Object')
  
"""
  i += 1

  # Get all Followers of current user
  temp_followers_df = get_followers(id_cur)
  temp_followers_df['source'] = temp_followers_df['id'] # Keep Original 'id' to remind all other fields are associated with Source (not Destination)
  temp_followers_df['destination'] = id_cur
  temp_followers_df['username'] = temp_followers_df['username'].astype('string')
  temp_followers_df['username'] = temp_followers_df['username'].str.strip()
  temp_followers_df['name'] = temp_followers_df['name'].astype('string')
  temp_followers_df['name'] = temp_followers_df['name'].str.strip()
  temp_followers_df.head()
  temp_followers_df.info()

  # Save
  filename_follower_out = f'twitter_expand_follower_{id_cur}.csv'
  temp_followers_df.to_csv(filename_follower_out, index=False)

  # Delay for Twitter API v2 limit of 1500 requests/15min period (+15 sec buffer)
  delay_sec = twitter_api2_delay_min*60 + 15
  print(f'  SLEEPING: {twitter_api2_delay_min} minutes (Twitter API v2 limit)')
  time.sleep(delay_sec)
"""

There are #1400 Followers


  First 5: ['Abdelra33210362', 'alexanderubaldo', 'levivoelz', 'RealzSyahril', 'jossromar36']


Processing #0:
  id: 1098521194305933312
  username: Abdelra33210362
  index: 438
200
    Appending row:           username     name  verified                   id description  \
0  Abdelra33210362  HIM2020     False  1098521194305933312               

   protected                created_at  \
0       True  2019-02-21T09:53:57.000Z   

                                   profile_image_url  \
0  https://abs.twimg.com/sticky/default_profile_i...   

   public_metrics.followers_count  public_metrics.following_count  \
0                               0                             307   

   public_metrics.tweet_count  public_metrics.listed_count  
0                           8                            0  


Processing #1:
  id: 256669500
  username: alexanderubaldo
  index: 956
200
    Appending row:    verified                                  profile_image_url  \


"\n  i += 1\n\n  # Get all Followers of current user\n  temp_followers_df = get_followers(id_cur)\n  temp_followers_df['source'] = temp_followers_df['id'] # Keep Original 'id' to remind all other fields are associated with Source (not Destination)\n  temp_followers_df['destination'] = id_cur\n  temp_followers_df['username'] = temp_followers_df['username'].astype('string')\n  temp_followers_df['username'] = temp_followers_df['username'].str.strip()\n  temp_followers_df['name'] = temp_followers_df['name'].astype('string')\n  temp_followers_df['name'] = temp_followers_df['name'].str.strip()\n  temp_followers_df.head()\n  temp_followers_df.info()\n\n  # Save\n  filename_follower_out = f'twitter_expand_follower_{id_cur}.csv'\n  temp_followers_df.to_csv(filename_follower_out, index=False)\n\n  # Delay for Twitter API v2 limit of 1500 requests/15min period (+15 sec buffer)\n  delay_sec = twitter_api2_delay_min*60 + 15\n  print(f'  SLEEPING: {twitter_api2_delay_min} minutes (Twitter API v2 lim

In [ ]:
# public_metrics.followers_count	public_metrics.following_count	public_metrics.tweet_count	public_metrics.listed_count

all_user_df.rename(columns={'public_metrics.followers_count': 'followers_count', 
                            'public_metrics.following_count': 'following_count',
                            'public_metrics.tweet_count': 'tweet_count',
                            'public_metrics.listed_count': 'listed_count',
                            }, inplace=True)

In [ ]:
all_user_df.head()

,username,name,verified,id,description,protected,created_at,profile_image_url,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,url,pinned_tweet_id,location,entities.url.urls,entities.description.hashtags,entities.description.mentions
0,Abdelra33210362,HIM2020,False,1098521194305933312,,True,2019-02-21T09:53:57.000Z,https://abs.twimg.com/sticky/default_profile_i...,0,307,8,0,NaN,NaN,NaN,NaN,NaN,NaN
1,alexanderubaldo,Alexander Ubaldo Gutiérrez,False,256669500,🇵🇦 Panameño 🚈 Especialista en Transporte y Mov...,False,2011-02-23T20:48:54.000Z,https://pbs.twimg.com/profile_images/133691715...,624,2223,12631,2,https://t.co/FHUe9SAB9O,1582908039597674496,Panama City,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 78, 'end': 87, 'tag': 'Política'}, ...",NaN
2,levivoelz,levi,False,1599001218,I make tools for creatives like @derivvapp and...,False,2013-07-16T18:43:02.000Z,https://pbs.twimg.com/profile_images/124231434...,56,358,708,0,https://t.co/gZNOOvzEQM,1525606679601041408,"Sonoma County, CA","[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,"[{'start': 32, 'end': 42, 'username': 'derivva..."


#### Get Followers

**WARNING: This can be a VERY LONG running process**

In [ ]:
# Iteratively expand all followers by getting who THEY follow and are being followed by

twitter_api2_delay_min = 15

# Create a list of all UNIQUE Follower IDs (Source)
all_follower_ls = list(set(twitter_all_follower_df.source))
print(f'There are #{len(all_follower_ls)} Followers\n\n')
print(f'  First 5: {all_follower_ls[:5]}\n=====================================\n\n')

# for i, afollower in enumerate(all_follower_ls):    # Get all follower followers 
for i, afollower in enumerate(all_follower_ls[:3]):   # Test first 3 followers
  id_cur = afollower
  # TODO: Embed within try/except block
  username_cur = twitter_all_follower_df[twitter_all_follower_df['id']==afollower]['username'].iloc[0]
  index_cur = twitter_all_follower_df[twitter_all_follower_df['id']==afollower]['username'].index[0]

  print(f"Processing #{i}:\n  id: {id_cur}\n  username: {username_cur}\n  index: {index_cur}")

  # Get all Followers of current user
  temp_followers_df = get_followers(id_cur)
  temp_followers_df['source'] = temp_followers_df['id'] # Keep Original 'id' to remind all other fields are associated with Source (not Destination)
  temp_followers_df['destination'] = id_cur
  temp_followers_df['username'] = temp_followers_df['username'].astype('string')
  temp_followers_df['username'] = temp_followers_df['username'].str.strip()
  temp_followers_df['name'] = temp_followers_df['name'].astype('string')
  temp_followers_df['name'] = temp_followers_df['name'].str.strip()
  temp_followers_df.head()
  temp_followers_df.info()

  # Save
  filename_follower_out = f'twitter_expand_follower_{id_cur}.csv'
  temp_followers_df.to_csv(filename_follower_out, index=False)

  # Delay for Twitter API v2 limit of 1500 requests/15min period (+15 sec buffer)
  delay_sec = twitter_api2_delay_min*60 + 15
  print(f'  SLEEPING: {twitter_api2_delay_min} minutes (Twitter API v2 limit)')
  time.sleep(delay_sec)



There are #1400 Followers


  First 5: [1224621238800146432, 883312305588535297, 1365422929652154370, 1369595759897829379, 1349661595018772483]


Processing #0:
  id: 1224621238800146432
  username: LuxInABottle
  index: 1136
200
200
Current users_ct: 200
200
Current users_ct: 300
200
Current users_ct: 319
200
Current users_ct: 338
200
Current users_ct: 357
200
Current users_ct: 376
200
Current users_ct: 395
200
Current users_ct: 414
200
Current users_ct: 433
200
Current users_ct: 452
200
Current users_ct: 471
200
Current users_ct: 490
200
Current users_ct: 509
200
Current users_ct: 528
429
Final users_ct:                       id                              name         username  \
0    1559793935589031936                     Scot Mennecke     MenneckeScot   
1    1561044369247019008                      Manish Prout      ManishProut   
2               97728804                       Ben Holfeld       BenHolfeld   
3    1560861711434907648                  Wilburn Hubertus  HubertusWi

,id,name,username,created_at,source,destination
0,1559793935589031936,Scot Mennecke,MenneckeScot,2022-08-17T06:47:11.000Z,1559793935589031936,1224621238800146432
1,1561044369247019008,Manish Prout,ManishProut,2022-08-20T17:36:01.000Z,1561044369247019008,1224621238800146432
2,97728804,Ben Holfeld,BenHolfeld,2009-12-18T19:08:57.000Z,97728804,1224621238800146432
3,1560861711434907648,Wilburn Hubertus,HubertusWilburn,2022-08-20T05:30:08.000Z,1560861711434907648,1224621238800146432
4,1425512767490977794,The Algorand Foundation Ltd ⬇️ 🌏,TheMyAlgorand,2021-08-11T17:41:49.000Z,1425512767490977794,1224621238800146432


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           528 non-null    object
 1   name         528 non-null    string
 2   username     528 non-null    string
 3   created_at   528 non-null    object
 4   source       528 non-null    object
 5   destination  528 non-null    int64 
dtypes: int64(1), object(3), string(2)
memory usage: 24.9+ KB
Processing #1:
  id: 883312305588535297
  username: jamesjdougherty
  index: 3
200


UnboundLocalError: ignored

## Get Following (run once for each gov)

Rate Limit: 15 Per App/15 Per User (within each 15min window)

(see https://developer.twitter.com/en/docs/twitter-api/rate-limits)

In [ ]:
def get_following(user_id):
    url = "https://api.twitter.com/2/users/{}/following".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    
    following_df = pd.json_normalize(json_response['data'])
    following_ct = following_df.shape[0]

    try:
        page_next_token = json_response['meta']['next_token']
    except:
        pass
    
    if not (page_next_token is None):
        
        while not (page_next_token is None):
            try:
                params = {"user.fields": "created_at", "pagination_token": page_next_token}
                json_response = connect_to_endpoint_with_params(url, params)
                my_following_df = pd.json_normalize(json_response['data'])
                
                following_df = pd.concat([following_df, my_following_df], ignore_index=True)
                following_ct += len(json_response['data'])
                print(f'Current users_ct: {following_ct}')
                
                try:
                    page_next_token = json_response['meta']['next_token']
                except:
                    pass
                      
            except:
                page_next_token = None
    
    print(f'Final users_ct: {following_ct}')
    
    return following_df

### Run ONCE for EACH Governor

In [ ]:
# Test

user_id = 2244994945
govs_d_following_df = get_following(user_id)
govs_d_following_df.head()
govs_d_following_df.info()

200
200
Current users_ct: 200
200
Current users_ct: 300
200
Current users_ct: 400
200
Current users_ct: 500
200
Current users_ct: 600
200
Current users_ct: 700
200
Current users_ct: 800
200
Current users_ct: 900
200
Current users_ct: 1000
200
Current users_ct: 1100
200
Current users_ct: 1200
200
Current users_ct: 1300
200
Current users_ct: 1400
429
Final users_ct: 1400


,name,username,id,created_at,withheld.country_codes
0,Elon Musk,elonmusk,44196397,2009-06-02T20:12:29.000Z,NaN
1,Aba Micah,2006legend,633050374,2012-07-11T16:54:40.000Z,NaN
2,Jessica Garson,jessicagarson,15772978,2008-08-08T02:16:23.000Z,NaN
3,Bitcoin Core Project,bitcoincoreorg,4839528448,2016-01-23T19:14:25.000Z,NaN
4,blue checkmark (real),Punkaboo,25029451,2009-03-18T04:43:46.000Z,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    1400 non-null   object
 1   username                1400 non-null   object
 2   id                      1400 non-null   object
 3   created_at              1400 non-null   object
 4   withheld.country_codes  1 non-null      object
dtypes: object(5)
memory usage: 54.8+ KB


In [ ]:
# Check not just repeating every 100

govs_d_following_df.iloc[0:10]
govs_d_following_df.iloc[95:105]

,name,username,id,created_at,withheld.country_codes
0,Elon Musk,elonmusk,44196397,2009-06-02T20:12:29.000Z,NaN
1,Aba Micah,2006legend,633050374,2012-07-11T16:54:40.000Z,NaN
2,Jessica Garson,jessicagarson,15772978,2008-08-08T02:16:23.000Z,NaN
3,Bitcoin Core Project,bitcoincoreorg,4839528448,2016-01-23T19:14:25.000Z,NaN
4,blue checkmark (real),Punkaboo,25029451,2009-03-18T04:43:46.000Z,NaN
5,cora,coraF256,1009484430250053632,2018-06-20T17:13:38.000Z,NaN
6,Amir Shevat,ashevat,108209516,2010-01-25T05:56:22.000Z,NaN
7,Sheryl Klein Lavi🏴󠁧󠁢󠁳󠁣󠁴󠁿,TheSherylKlein,1062359582,2013-01-05T06:56:49.000Z,NaN
8,Liliana Aidé Monge,mongeliliana,243665363,2011-01-27T15:10:03.000Z,NaN
9,julie✨,JulieMendoza206,459860328,2012-01-10T04:01:22.000Z,NaN


,name,username,id,created_at,withheld.country_codes
95,オーウェン,DarylOwenJ,839668676,2012-09-22T12:30:57.000Z,NaN
96,Hatice Edis 📟,edisdev,618029513,2012-06-25T11:52:19.000Z,NaN
97,Ian Kirker,ikirker,1934669329,2013-10-04T15:46:40.000Z,NaN
98,Jonathan Laurin,jolaurin24,194811144,2010-09-25T01:35:35.000Z,NaN
99,Alfonso 🍀🪐,alfongj,38292395,2009-05-06T22:21:23.000Z,NaN
100,Serdar Kiliç,serdar,5177,2006-09-02T01:59:11.000Z,NaN
101,Chima,chmanw,1057825998,2013-01-03T13:52:19.000Z,NaN
102,liam,inky,13148,2006-11-20T00:04:50.000Z,NaN
103,Jonathan,jony_silva,15399427,2008-07-12T02:02:26.000Z,NaN
104,Simon,simon_jthompson,534682665,2012-03-23T18:32:49.000Z,NaN


In [ ]:
govs_d_following_df.tail()

,name,username,id,created_at,withheld.country_codes
1395,Omar.,_oememartinez,1663613648,2013-08-11T22:39:34.000Z,NaN
1396,AR,dr_turkleberry,113731124,2010-02-12T21:10:53.000Z,NaN
1397,sarathdr,sarathdr,14273442,2008-04-01T09:01:23.000Z,NaN
1398,Patxi Bocos,patxibocos,260117096,2011-03-03T06:50:40.000Z,NaN
1399,Iordanis Giannakakis,iordanis_g,95617311,2009-12-09T10:17:22.000Z,NaN


In [ ]:
# Insert user_id as 'source' in first column
#        rename following id as 'destination'

govs_d_following_df.insert(0, 'source', user_id)
govs_d_following_df.rename(columns={'id':'destination'}, inplace=True)

In [ ]:
# Verify
 
govs_d_following_df.head()
govs_d_following_df.info()

,source,name,username,destination,created_at,withheld.country_codes
0,2244994945,Elon Musk,elonmusk,44196397,2009-06-02T20:12:29.000Z,NaN
1,2244994945,Aba Micah,2006legend,633050374,2012-07-11T16:54:40.000Z,NaN
2,2244994945,Jessica Garson,jessicagarson,15772978,2008-08-08T02:16:23.000Z,NaN
3,2244994945,Bitcoin Core Project,bitcoincoreorg,4839528448,2016-01-23T19:14:25.000Z,NaN
4,2244994945,blue checkmark (real),Punkaboo,25029451,2009-03-18T04:43:46.000Z,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   source                  1400 non-null   int64 
 1   name                    1400 non-null   object
 2   username                1400 non-null   object
 3   destination             1400 non-null   object
 4   created_at              1400 non-null   object
 5   withheld.country_codes  1 non-null      object
dtypes: int64(1), object(5)
memory usage: 65.8+ KB


In [ ]:
# Save

filename_out = f'twitter_following_{user_id}.csv'
govs_d_following_df.to_csv(filename_out, index=False)

In [ ]:
# Make dummy copies to test DataFile Concatenation

!cp $filename_out twitter_following_12345.csv

!cp $filename_out twitter_following_23456.csv

!ls -al twitter_following_*.csv

-rw-r--r-- 1 root root 102186 Nov 18 12:49 twitter_following_12345.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:46 twitter_following_2244994945.csv
-rw-r--r-- 1 root root 102186 Nov 18 12:49 twitter_following_23456.csv


### FINALLY: Concatenate all Following DataFrames and Save to ONE *.csv File

In [ ]:
datafile_csv_ls = glob.glob('twitter_following_*.csv')

all_df = pd.DataFrame()

for i, adatafile in enumerate(datafile_csv_ls):
    print(f'Processing #{i}: {adatafile}')
    current_df = pd.read_csv(adatafile, index_col=[0])
    print(f'      shape: {current_df.shape}')
    if i == 0:
        all_df = current_df.copy(deep=True)
    else:
        all_df = pd.concat([all_df, current_df], ignore_index=True)
        
print(f'Combined DataFrame.shape: {all_df.shape}')
    

Processing #0: twitter_following_2244994945.csv
      shape: (1400, 5)
Processing #1: twitter_following_23456.csv
      shape: (1400, 5)
Processing #2: twitter_following_12345.csv
      shape: (1400, 5)
Combined DataFrame.shape: (4200, 5)


In [ ]:
# Save Combined DataFrame

datetime_str = time.strftime("%Y%m%d-%H%M%S")

filename_out = f'twitter_all_following_{datetime_str}.csv'

try:
  all_df.to_csv(filename_out, index=False)
  print(f'Saved to: {filename_out}')
except:
  print(f'ERROR: Cannot save to: {filename_out}')

Saved to: twitter_all_following_20221118-125029.csv


In [ ]:
# Verify

!head -n 10 $filename_out

name,username,destination,created_at,withheld.country_codes
Elon Musk,elonmusk,44196397,2009-06-02T20:12:29.000Z,
Aba Micah,2006legend,633050374,2012-07-11T16:54:40.000Z,
Jessica Garson,jessicagarson,15772978,2008-08-08T02:16:23.000Z,
Bitcoin Core Project,bitcoincoreorg,4839528448,2016-01-23T19:14:25.000Z,
blue checkmark (real),Punkaboo,25029451,2009-03-18T04:43:46.000Z,
cora,coraF256,1009484430250053632,2018-06-20T17:13:38.000Z,
Amir Shevat,ashevat,108209516,2010-01-25T05:56:22.000Z,
Sheryl Klein Lavi🏴󠁧󠁢󠁳󠁣󠁴󠁿,TheSherylKlein,1062359582,2013-01-05T06:56:49.000Z,
Liliana Aidé Monge,mongeliliana,243665363,2011-01-27T15:10:03.000Z,


## Expand Followers

In [ ]:
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
all_df                DataFrame                           na<...>\n[4200 rows x 5 columns]
current_df            DataFrame                             <...>\n[1400 rows x 5 columns]
followers_df          DataFrame                             <...>n\n[100 rows x 4 columns]
govs_d_df             DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_followers_df   DataFrame          destination        <...>\n[1400 rows x 5 columns]
govs_d_following_df   DataFrame              source         <...>\n[1400 rows x 6 columns]
govs_d_user_df        DataFrame                            u<...>                    NaN  
govs_df               DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df             DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df        DataFrame    Empty DataFrame\nColumns:<...> listed_count]\nIndex: []


In [ ]:
for i, afollower in enumerate()

## Expand Following

## Get User Timeline

In [ ]:
def get_user_timeline(id, params):
    url = "https://api.twitter.com/2/users/{}/tweets".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# Tweet fields are adjustable.
# Options include:
# attachments, author_id, context_annotations,
# conversation_id, created_at, entities, geo, id,
# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
# source, text, and withheld
my_params = {"tweet.fields": "created_at"}

# Test
user_id = 2244994945

json_response = get_user_timeline(user_id, my_params)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_timeline_df = pd.DataFrame(json_response['data'])
my_timeline_df.head()
my_timeline_df.info()

200
{
    "data": [
        {
            "created_at": "2022-11-02T23:15:29.000Z",
            "edit_history_tweet_ids": [
                "1587946527955329024"
            ],
            "id": "1587946527955329024",
            "text": "As always, we\u2019re just a Tweet away, so feel free to reach out with any questions. We\u2019re grateful for your partnership to #BuildWhatsNext"
        },
        {
            "created_at": "2022-11-02T23:15:29.000Z",
            "edit_history_tweet_ids": [
                "1587946526617264128"
            ],
            "id": "1587946526617264128",
            "text": "We\u2019ll still celebrate the soon-to-be-announced winners of our Chirp Developer Challenge - stay tuned for more details!"
        },
        {
            "created_at": "2022-11-02T23:15:28.000Z",
            "edit_history_tweet_ids": [
                "1587946525245816832"
            ],
            "id": "1587946525245816832",
            "text": "We\u2019re currently hard at

,text,edit_history_tweet_ids,created_at,id
0,"As always, we’re just a Tweet away, so feel fr...",[1587946527955329024],2022-11-02T23:15:29.000Z,1587946527955329024
1,We’ll still celebrate the soon-to-be-announced...,[1587946526617264128],2022-11-02T23:15:29.000Z,1587946526617264128
2,We’re currently hard at work to make Twitter b...,[1587946525245816832],2022-11-02T23:15:28.000Z,1587946525245816832
3,💡 #TipTuesday: Ever wondered how to get the v...,[1587519847281397767],2022-11-01T19:00:00.000Z,1587519847281397767
4,✍️Fill in the blank ⬇️\n\nI start my morning o...,[1587066866824085505],2022-10-31T13:00:01.000Z,1587066866824085505


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    10 non-null     object
 1   edit_history_tweet_ids  10 non-null     object
 2   created_at              10 non-null     object
 3   id                      10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


## Get Liked Tweets

In [ ]:
def get_liked_tweets(id, params):
    url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# Tweet fields are adjustable.
# Options include:
# attachments, author_id, context_annotations,
# conversation_id, created_at, entities, geo, id,
# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
# source, text, and withheld
tweet_fields = "tweet.fields=lang,author_id"
# Be sure to replace your-user-id with your own user ID or one of an authenticating user
# You can find a user ID by using the user lookup endpoint
id = "your-user-id"
# You can adjust ids to include a single Tweets.
# Or you can add to up to 100 comma-separated IDs
url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)

# Test
user_id = 2244994945

json_response = get_liked_tweets(user_id, tweet_fields)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_liked_tweets_df = pd.DataFrame(json_response['data'])
my_liked_tweets_df.head()
my_liked_tweets_df.info() 

200
{
    "data": [
        {
            "author_id": "1136175005060878337",
            "edit_history_tweet_ids": [
                "1581308268122877953"
            ],
            "id": "1581308268122877953",
            "lang": "en",
            "text": "@TwitterDev Tips and tricks would be cool!!"
        },
        {
            "author_id": "109628138",
            "edit_history_tweet_ids": [
                "1575158698225258500"
            ],
            "id": "1575158698225258500",
            "lang": "en",
            "text": "On a call with @TwitterDev team looking at the upcoming platform functionality \u2014 some exiting functionality in the works. \n\nAlso, I \u2764\ufe0f the approach twitter is taking to feature development. Good job, bird ppl"
        },
        {
            "author_id": "1120050519182016513",
            "edit_history_tweet_ids": [
                "1575117901647986691"
            ],
            "id": "1575117901647986691",
            "lang": "en",


,text,id,lang,edit_history_tweet_ids,author_id
0,@TwitterDev Tips and tricks would be cool!!,1581308268122877953,en,[1581308268122877953],1136175005060878337
1,On a call with @TwitterDev team looking at the...,1575158698225258500,en,[1575158698225258500],109628138
2,Big thanks @TwitterDev for these wonderful hig...,1575117901647986691,en,[1575117901647986691],1120050519182016513
3,Congratulations to @triketora and the @blockpa...,1572733815965253632,en,[1572733815965253632],108209516
4,"🎉 Thank you so much for the kind gesture, @Twi...",1571949434010992640,en,[1571949434010992640],1264433760


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    97 non-null     object
 1   id                      97 non-null     object
 2   lang                    97 non-null     object
 3   edit_history_tweet_ids  97 non-null     object
 4   author_id               97 non-null     object
dtypes: object(5)
memory usage: 3.9+ KB


## Get Liking Users

In [ ]:
json_response = get_liking_users(user_id, user_fields)
pprint(json_response)

200
{'meta': {'next_token': '7140dibdnow9c7btw420jlmkll17j0clkx1bl4w5gi6nr',
          'result_count': 0}}


In [ ]:
def get_liking_users(id, params):
    url = "https://api.twitter.com/2/tweets/{}/liking_users".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
user_fields = "user.fields=created_at,description"
# You can replace the ID given with the Tweet ID you wish to like.
# You can find an ID by using the Tweet lookup endpoint
id = "1590693373056122880"
# You can adjust ids to include a single Tweets.
# Or you can add to up to 100 comma-separated IDs

# Test
user_id = "1590693373056122880"

json_response = get_liking_users(user_id, user_fields)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_liking_users_df = pd.DataFrame(json_response['data'])
my_liking_users_df.head()
my_liking_users_df.info() 

200
{
    "data": [
        {
            "created_at": "2011-10-15T22:25:54.000Z",
            "description": "",
            "id": "391659128",
            "name": "Dimitri Schmid",
            "username": "dimitrischmid"
        },
        {
            "created_at": "2015-03-31T15:54:56.000Z",
            "description": "Programming Officer at ONGC!!!",
            "id": "3120779995",
            "name": "ABHISHEK BHARDWAJ",
            "username": "abhishek21795"
        },
        {
            "created_at": "2020-04-29T19:27:35.000Z",
            "description": "",
            "id": "1255579413934030848",
            "name": "devesh_deepak",
            "username": "deveshdeepak9"
        },
        {
            "created_at": "2018-11-29T20:52:12.000Z",
            "description": "I enable insights using data and math. Builder of the contact index. AI consultant for German federal government. Views and titles (Dr. habil.) my own.",
            "id": "1068246270752538630",
     

,id,name,description,username,created_at
0,391659128,Dimitri Schmid,,dimitrischmid,2011-10-15T22:25:54.000Z
1,3120779995,ABHISHEK BHARDWAJ,Programming Officer at ONGC!!!,abhishek21795,2015-03-31T15:54:56.000Z
2,1255579413934030848,devesh_deepak,,deveshdeepak9,2020-04-29T19:27:35.000Z
3,1068246270752538630,Sten Rüdiger,I enable insights using data and math. Builder...,StenRuediger,2018-11-29T20:52:12.000Z
4,3319164733,Ifty Mohammad Rezwan,"Data Monger,\nAll opinions are my own.",imr165,2015-08-18T20:34:13.000Z


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           93 non-null     object
 1   name         93 non-null     object
 2   description  93 non-null     object
 3   username     93 non-null     object
 4   created_at   93 non-null     object
dtypes: object(5)
memory usage: 3.8+ KB


# END OF NOTEBOOK

## Appendix: Set Union

In [ ]:
# Find common elements in 2 data collections (here lists)
# 
aoc_ls = [1,2,3,4,5,6,7,8,9,10]
mgt_ls = [8,9,10,11,12,13,14,15,16,17,18]

common_ls = list(set(aoc_ls).intersection(set(mgt_ls)))

[8, 9, 10]

In [ ]:
for auser in common_ls:
    user (user_fields=['created_at','public_metrics'])

## Appendix: Join csv

In [ ]:
aoc_followers_df = pd.read_csv('aoc_following_ids.csv', index_col=[0])
aoc_followers_df.head()
aoc_followers_df.info()

,id,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
NaN,13479,NaN,NaN,NaN,NaN,NaN
NaN,21619519,NaN,NaN,NaN,NaN,NaN
NaN,1391845341641773067,NaN,NaN,NaN,NaN,NaN
NaN,1051837159639007233,NaN,NaN,NaN,NaN,NaN
NaN,136550204,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Float64Index: 3939 entries, nan to nan
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          3939 non-null   int64  
 1   Unnamed: 2  3 non-null      object 
 2   Unnamed: 3  0 non-null      float64
 3   Unnamed: 4  0 non-null      float64
 4   Unnamed: 5  0 non-null      float64
 5   Unnamed: 6  0 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 215.4+ KB


In [ ]:
mtg_followers_df = pd.read_csv('mtg_following_ids.csv', index_col=[0])
mtg_followers_df.head()
mtg_followers_df.info()

,id,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
NaN,935942321756966912,NaN,NaN,NaN,NaN,NaN
NaN,902200087,NaN,NaN,NaN,NaN,NaN
NaN,214924095,NaN,NaN,NaN,NaN,NaN
NaN,1565070900243873792,NaN,NaN,NaN,NaN,NaN
NaN,44196397,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Float64Index: 275 entries, nan to nan
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          275 non-null    int64  
 1   Unnamed: 2  0 non-null      float64
 2   Unnamed: 3  0 non-null      float64
 3   Unnamed: 4  0 non-null      float64
 4   Unnamed: 5  0 non-null      float64
 5   Unnamed: 6  0 non-null      float64
dtypes: float64(5), int64(1)
memory usage: 15.0 KB


In [ ]:
aoc_followers_ls = aoc_followers_df.id.to_list()
len(aoc_followers_ls)
aoc_followers_ls[:5]


3939

[13479, 21619519, 1391845341641773067, 1051837159639007233, 136550204]

In [ ]:
mtg_followers_ls = mtg_followers_df.id.to_list()
len(mtg_followers_ls)
mtg_followers_ls[:5]

275

[935942321756966912, 902200087, 214924095, 1565070900243873792, 44196397]

In [ ]:
aoc_mtg_followers_ls = list(set(aoc_followers_ls).intersection(set(mtg_followers_ls)))
len(aoc_mtg_followers_ls)
aoc_mtg_followers_ls

8

[2916305152, 18208354, 975200486, 17685258, 12, 963480595, 15207668, 353890966]

## Appendix: Junk

In [ ]:
# Twitter Standard API v1.1
# Get Tweet Timelines
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

api.user_timeline('GovEvers')

In [ ]:
govs_tweets_df = pd.DataFrame()

for i, agov_username in enumerate(govs_username_ls[:3]):
    print(f'Processing #{i}: {agov_username}')
    get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
gc.collect()
    # govs_tweets_df.append(agov_username)
    # govs_df.apply(lambda x:get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
    df.apply(lambda x:get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
gc.collect()
    gc.collect()

Processing #0: GovEvers


26

Processing #1: TinaKotek


0

Processing #2: GovSisolak


0